In [2]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
seaborn.__version__

'0.9.0'

In [11]:
# Leer los datos
dataset = pandas.read_csv('data/the-human-freedom-index-hfi_cc_2018.csv')
print('Tamaño de la tabla: {} filas, {} columnas'.format(dataset.shape[0], dataset.shape[1]))
print('Muestra de la tabla:')
dataset.head(3)

Tamaño de la tabla: 1458 filas, 123 columnas
Muestra de la tabla:


,year,ISO_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,2016,ALB,Albania,Eastern Europe,6.661503,4.547244,4.666508,5.291752,8.920429,10.0,...,4.050196,7.324582,7.074366,6.705863,6.906901,7.54,34.0,7.568140,48.0,2.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,NaN,NaN,3.819566,9.456254,10.0,...,3.765515,8.523503,7.029528,5.676956,5.268992,4.99,159.0,5.135886,155.0,4.0
2,2016,AGO,Angola,Sub-Saharan Africa,NaN,NaN,NaN,3.451814,8.060260,5.0,...,1.945540,8.096776,6.782923,4.930271,5.518500,5.17,155.0,5.640662,142.0,4.0


In [15]:
#Atender a las columnas que son importantes para este estudio
important_cols = ['year', 'ISO_code', 'countries', 'region']
important_cols += [col for col in dataset.columns if 'pf_identity' in col]
important_cols += [
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]
important_dataset = dataset[important_cols]
print('Muestra de la parte que nos interesa de la tabla:')
important_dataset.head(3)

Muestra de la parte que nos interesa de la tabla:


,year,ISO_code,countries,region,pf_identity_legal,pf_identity_parental_marriage,pf_identity_parental_divorce,pf_identity_parental,pf_identity_sex_male,pf_identity_sex_female,pf_identity_sex,pf_identity_divorce,pf_identity,ef_score,ef_rank,hf_score,hf_rank
0,2016,ALB,Albania,Eastern Europe,0.0,10.0,10.0,10.0,10.0,10.0,10.0,5.0,6.250000,7.54,34.0,7.568140,48.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,0.0,5.0,2.5,0.0,0.0,0.0,0.0,0.833333,4.99,159.0,5.135886,155.0
2,2016,AGO,Angola,Sub-Saharan Africa,10.0,10.0,10.0,10.0,0.0,0.0,0.0,10.0,7.500000,5.17,155.0,5.640662,142.0


# 1. Estadísticos descriptivos

## 1.1 Calcular el rango de las variables.

In [32]:
# print(important_dataset.dtypes)
print('Rangos de las variables')
for col in important_cols:
    serie = important_dataset[col]
    col_type = serie.dtype
    if col_type in ['float64', 'int64']:
        max_value, min_value = serie.max(), serie.min()
        print('  -- {}. Maximo: {}. Mínimo: {}. Rango: {}'.format(col, max_value, min_value, max_value - min_value))

Rangos de las variables
  -- year. Maximo: 2016. Mínimo: 2008. Rango: 8
  -- pf_identity_legal. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_parental_marriage. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_parental_divorce. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_parental. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_sex_male. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_sex_female. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_sex. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity_divorce. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- pf_identity. Maximo: 10.0. Mínimo: 0.0. Rango: 10.0
  -- ef_score. Maximo: 9.19. Mínimo: 2.88. Rango: 6.31
  -- ef_rank. Maximo: 162.0. Mínimo: 1.0. Rango: 161.0
  -- hf_score. Maximo: 9.126312939. Mínimo: 3.765826877. Rango: 5.360486062
  -- hf_rank. Maximo: 162.0. Mínimo: 1.0. Rango: 161.0


## 1.2 Obtener media, mediana y desviación estándar de las variables pf_identity y hf_score en el mundo y compararla con la de Latinoamérica y el caribe. ¿Tiene sentido calcular la moda?

## 1.3 ¿Son todos los valores de pf_identity y hf_score directamente comparables? ¿Qué otra variable podría influenciarlos?

## 1.4 ¿Cómo pueden sanearse los valores faltantes?

## 1.5 ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? ¿Los outliers, son globales o por grupo? ¿Los eliminaría del conjunto de datos?